# Symbolic math

**Table of contents**<a id='toc0_'></a>    
- 1. [Solve consumer problem](#toc1_)    
  - 1.1. [Latex](#toc1_1_)    
  - 1.2. [Turn solution into Python function](#toc1_2_)    
  - 1.3. [Analyzing properties of the solution (expression)](#toc1_3_)    
- 2. [More features of symbolic math (mixed goodies)](#toc2_)    
- 3. [Numerical vs. symbolic solution](#toc3_)    
- 4. [Solving matrix equations symbolically (+)](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

You will learn to use symbolic math with **sympy** to solve equations.

**sympy:** [overview](https://docs.sympy.org/latest/index.html) + [tutorial](https://docs.sympy.org/latest/tutorial/index.html#tutorial)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

from scipy import optimize

import sympy as sm
from IPython.display import display

## 1. <a id='toc1_'></a>[Solve consumer problem](#toc0_)

Consider solving the following problem:

$$ 
\max_{x_1,x_2} x_1^{\alpha} x_2^{\beta} \text{ s.t. } p_1x_1 + p_2x_2 = m
$$

Define all symbols:

In [2]:
x1 = sm.symbols('x_1') # x1 is a Python variable representing the symbol x_1
x2 = sm.symbols('x_2')
alpha = sm.symbols('alpha')
beta = sm.symbols('beta')
p1 = sm.symbols('p_1')
p2 = sm.symbols('p_2')
m = sm.symbols('m')

print('x1 is of type: ', type(x1))

x1 is of type:  <class 'sympy.core.symbol.Symbol'>


Define objective and budget constraint:

In [3]:
# write out the equation as if it was regular code
objective = x1**alpha*x2**beta
objective

x_1**alpha*x_2**beta

In [4]:
# define the budget constraint as an equality
budget_constraint = sm.Eq(p1*x1+p2*x2,m)
budget_constraint

Eq(p_1*x_1 + p_2*x_2, m)

Solve in **four steps**:

1. **Isolate** $x_2$ from the budget constraint
2. **Substitute** in $x_2$
3. **Take the derivative** wrt. $x_1$
4. **Solve the FOC** for $x_1$

**Step 1: Isolate**

In [5]:
# isolate x2 on LHS
x2_from_con = sm.solve(budget_constraint, x2)
x2_from_con[0]

(m - p_1*x_1)/p_2

**Step 2: Substitute**

In [6]:
objective_subs = objective.subs(x2, x2_from_con[0])
objective_subs

x_1**alpha*((m - p_1*x_1)/p_2)**beta

**Step 3: Take the derivative**

In [7]:
foc = sm.diff(objective_subs, x1)
foc

alpha*x_1**alpha*((m - p_1*x_1)/p_2)**beta/x_1 - beta*p_1*x_1**alpha*((m - p_1*x_1)/p_2)**beta/(m - p_1*x_1)

**Step 4: Solve the FOC**

In [8]:
sol = sm.solve(sm.Eq(foc,0), x1)
sol[0]

alpha*m/(p_1*(alpha + beta))

> An alternative is `sm.solveset()`, which will be the default in the future, but it is still a bit immature in my view.

**Task:** Solve the consumer problem with quasi-linear preferences,

$$ \max_{x_1,x_2} \sqrt{x_1} + \gamma x_2 \text{ s.t. } p_1x_1 + p_2x_2 = m $$

In [9]:
# write your code here        

**Solution:**

In [10]:
gamma = sm.symbols('gamma')
objective_alt = sm.sqrt(x1) + gamma*x2
objective_alt_subs = objective_alt.subs(x2,x2_from_con[0])
foc_alt = sm.diff(objective_alt_subs,x1)
sol_alt = sm.solve(foc_alt,x1)
sol_alt[0]

p_2**2/(4*gamma**2*p_1**2)

### 1.1. <a id='toc1_1_'></a>[Latex](#toc0_)

**LaTex:** Print in LaTex format:

In [11]:
print(sm.latex(sol[0]))

\frac{\alpha m}{p_{1} \left(\alpha + \beta\right)}


### 1.2. <a id='toc1_2_'></a>[Turn solution into Python function](#toc0_)

Sympy can do a fantastic trick! Once you have the solution of your equation, this can be **turned into a Python function**. Thus you can use the solution on arrays. It's called lambdification (think "lambda functions").

In [12]:
# simple example. 
# 1st element of lambdify: a tuple of symbols to be used. 
# 2nd element: the expression used on the symbols.  

x = sm.symbols('x')
x_square = sm.lambdify(args = (x), expr = x**2)
x_square(12)

144

In [13]:
# create a function out of the solution by providing the "expression" you want 
# (ie the solution) and the inputs to the expression in a tuple. 

sol_func = sm.lambdify(args=(p1,m,alpha,beta),expr=sol[0])

# run solution. DO NOT overwrite the SYMBOLS (m,alpha,beta) with numeric data
p1_vec = np.array([1.2,3,5,9])
m_val = 10
alpha_val = 0.5
beta_val = 0.5

# run solution function with vector of prices
demand_p1 = sol_func(p1_vec, m_val, alpha_val, beta_val)

for d in demand_p1:
    print(f'demand: {d:1.3f}')

demand: 4.167
demand: 1.667
demand: 1.000
demand: 0.556


**Task:** Solve the consumer problem with quasi-linear preferences numerically with $\gamma=0.5,p_1=2,p_2=1,m=10$.

In [14]:
# write your code here 

**Solution:**

In [15]:
gamma_val = 0.5
p1_val = 2.0
p2_val = 1.0
m_val = 10.0
sol_alt_func = sm.lambdify(args=(p1,p2,m,gamma),expr=sol_alt[0])
demand = sol_alt_func(p1_val,p2_val,m_val,gamma_val)
print(f'demand: {demand:1.3f}')

demand: 0.250


### 1.3. <a id='toc1_3_'></a>[Analyzing properties of the solution (expression)](#toc0_)

**Is demand always positive?**

Give the computer the **information** we have. I.e. that $p_1$, $p_2$, $\alpha$, $\beta$, $m$ are all strictly positive:

In [16]:
for var in [p1,p2,alpha,beta,m]:
    sm.assumptions.assume.global_assumptions.add(sm.Q.positive(var)) # var is always positive
sm.assumptions.assume.global_assumptions    

{Q.positive(alpha),
 Q.positive(beta),
 Q.positive(m),
 Q.positive(p_1),
 Q.positive(p_2)}

**Ask** the computer a **question**:

In [17]:
answer = sm.ask(sm.Q.positive(sol[0]))
print(answer)

True


We need the assumption that $p_1 > 0$:

In [18]:
sm.assumptions.assume.global_assumptions.remove(sm.Q.positive(p1))
answer = sm.ask(sm.Q.positive(sol[0]))
print(answer)

None


To clear all assumptions we can use:

In [19]:
sm.assumptions.assume.global_assumptions.clear()

## 2. <a id='toc2_'></a>[More features of symbolic math (mixed goodies)](#toc0_)

In [20]:
x = sm.symbols('x')

**Derivatives:** Higher order derivatives are also available

In [21]:
sm.Derivative('x**4',x,x)

Derivative(x**4, (x, 2))

In [22]:
sm.Derivative('x**4',x,x,evaluate=True)

12*x**2

Alternatively,

In [23]:
sm.diff('x**4',x,x)

12*x**2

In [24]:
expr = sm.Derivative('x**4',x,x)
expr.doit()

12*x**2

With two variables:

In [25]:
y = sm.symbols('y')
sm.diff('(x**2)*log(y)*exp(y)',x,y)

2*x*(log(y) + 1/y)*exp(y)

**Integrals:**

In [26]:
sm.Integral(sm.exp(-x), (x, 0, sm.oo))

Integral(exp(-x), (x, 0, oo))

In [27]:
sm.integrate(sm.exp(-x), (x, 0, sm.oo))

1

**Limits:**

In [28]:
c = sm.symbols('c')
rho = sm.symbols('rho')

# Write up the definition of your limit
sm.Limit((c**(1-rho)-1)/(1-rho),rho,1)

Limit((c**(1 - rho) - 1)/(1 - rho), rho, 1)

In [29]:
# Evaluate the limit
sm.limit((c**(1-rho)-1)/(1-rho),rho,1)

log(c)

**Integers:**

In [30]:
X = sm.Integer(7)/sm.Integer(3)
Y = sm.Integer(3)/sm.Integer(8)
display(X)
display(Y)
Z = 3
(X*Y)**Z

7/3

3/8

343/512

**Simplify:**

In [31]:
expr = sm.sin(x)**2 + sm.cos(x)**2
display(expr)

sin(x)**2 + cos(x)**2

In [32]:
sm.simplify(expr)

1

**Solve multiple equations at once:**

In [33]:
x = sm.symbols('x')
y = sm.symbols('y')
Eq1 = sm.Eq(x**2+y-2,0)
Eq2 = sm.Eq(y**2-4,0)
display(Eq1)
display(Eq2)

Eq(x**2 + y - 2, 0)

Eq(y**2 - 4, 0)

In [34]:
# Solve the system
sol = sm.solve([Eq1,Eq2],[x,y])

# print all solutions
for xy in sol:
    print(f'(x,y) = ({xy[0]},{xy[1]})')

(x,y) = (-2,-2)
(x,y) = (0,2)
(x,y) = (0,2)
(x,y) = (2,-2)


## 3. <a id='toc3_'></a>[Numerical vs. symbolic solution](#toc0_)

$$f(x) = x^2 - 1$$

**Numiercal solution:** Potential problems include

1. Convergence to only one root
2. Convergence not exact
3. No convergence

In [35]:
for x0 in [-10,-5,0,5,10]:
    result = optimize.root(lambda x: x**2-1,x0)
    print(f'{x0 = :3d} implies {result.x[0]:19.16f} [{result.success = :}]')

x0 = -10 implies -1.0000000000000000 [result.success = True]
x0 =  -5 implies -1.0000000000000004 [result.success = True]
x0 =   0 implies  0.0000000000000000 [result.success = False]
x0 =   5 implies  1.0000000000000004 [result.success = True]
x0 =  10 implies  1.0000000000000000 [result.success = True]


**Symbolic solution:** Find all solutions exactly

In [36]:
result = sm.solveset(x**2-1,)
display(result)

{-1, 1}

## 4. <a id='toc4_'></a>[Solving matrix equations symbolically (+)](#toc0_)

$$ Ax = b $$

**Construct matrix function:**

In [37]:
def construct_sympy_matrix(positions,name='a'):
    """ construct sympy matrix with non-zero elements in positions
    
    Args:
    
        Positions (list): list of positions in strings, e.g. ['11','31']
    
    Returns:
    
        mat (sympy.matrix): Sympy Matrix
    
    """
    
    # a. dictionary of element with position as key and a_position as value
    entries = {f'{ij}':sm.symbols(f'{name}_{ij}') for ij in positions}

    # b. function for creating element or zero
    add = lambda x: entries[x] if x in entries else 0

    # c. create matrix
    mat_as_list = [[add(f'{1+i}{1+j}') for j in range(3)] for i in range(3)]
    mat = sm.Matrix(mat_as_list)

    return mat

**Full matrix function:**

In [38]:
def fill_sympy_matrix(A_sm,A,name='a'):

    n,m = A.shape

    # a. make all substitution
    A_sm_copy = A_sm
    for i in range(n):
        for j in range(m):
            if not A[i,j] == 0:
                A_sm_copy = A_sm_copy.subs(f'{name}_{1+i}{1+j}',A[i,j])
    
    # b. lambdify with no inputs
    f = sm.lambdify((),A_sm_copy)

    # c. return filled matrix
    return f()

**Construct a symbolic matrix:**

In [39]:
A_sm = construct_sympy_matrix(['11','12','21','22','32','33'])
A_sm

Matrix([
[a_11, a_12,    0],
[a_21, a_22,    0],
[   0, a_32, a_33]])

**Find the inverse symbolically:**

In [40]:
A_sm_inv = A_sm.inv()
A_sm_inv

Matrix([
[               a_22/(a_11*a_22 - a_12*a_21),                -a_12/(a_11*a_22 - a_12*a_21),      0],
[              -a_21/(a_11*a_22 - a_12*a_21),                 a_11/(a_11*a_22 - a_12*a_21),      0],
[a_21*a_32/(a_11*a_22*a_33 - a_12*a_21*a_33), -a_11*a_32/(a_11*a_22*a_33 - a_12*a_21*a_33), 1/a_33]])

**Fill in the numeric values:**

In [41]:
A = np.array([[3.0,2.0,0.0],[1.0,-1.0,0],[0.0,5.0,1.0]])
b = np.array([2.0,4.0,-1.0])

In [42]:
A_inv_num = fill_sympy_matrix(A_sm_inv,A)
x = A_inv_num@b
print('solution:',x)

solution: [ 2. -2.  9.]


**Note:** The inverse multiplied by the determinant looks nicer

In [43]:
A_sm_det = A_sm.det()
A_sm_det

a_11*a_22*a_33 - a_12*a_21*a_33

In [44]:
A_sm_inv_raw = sm.simplify(A_sm_inv*A_sm_det)
A_sm_inv_raw

Matrix([
[ a_22*a_33, -a_12*a_33,                     0],
[-a_21*a_33,  a_11*a_33,                     0],
[ a_21*a_32, -a_11*a_32, a_11*a_22 - a_12*a_21]])